In [ ]:
import pandas as pd
import io
import requests
import datetime
import plotly.express as px

## Get the Timeseries Data

Add documentation about the time series here.

In [ ]:
from functions import get_time_series

confirmed, deaths, recovered, time_series_dates = get_time_series(local=True)

## Get all the Daily Reports 

Add documentation about these reports here.

In [ ]:
from functions import get_daily_reports

daily_report_data, daily_report_dates = get_daily_reports(local=True)

## All Done Gathering Data.

Let's merge the time_series_data into one dataframe

In [ ]:
from model import data_by_area
from data import us_confirmed 

data = data_by_area(area='Nebraska', col='State', df=us_confirmed)
data.plot()

In [ ]:

def data_by_area(area='US', col='Country/Region', df=None):
	from data import time_series_date_list
	print(col)
	data =pd.Series(
		[df.loc[(df[col] == area)][date].sum() for date in time_series_date_list],
		)
	return data


In [ ]:
data_by_area(area='US', df=confirmed).plot()

In [1]:
from functions import make_country_labels, make_state_labels

from data import confirmed

data = confirmed.copy()
print(data.groupby('Country/Region').sum().iloc[:,-1].sort_values(ascending=False).index)


make_country_labels(data=confirmed)

Getting daily reports using local data
Index(['Vietnam', 'French Guiana', 'Croatia', 'Cruise Ship', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Dominican Republic', 'Ecuador',
       ...
       'Mongolia', 'Morocco', 'Nepal', 'Netherlands', 'New Zealand', 'Nigeria',
       'North Macedonia', 'Norway', 'Oman', 'Afghanistan'],
      dtype='object', name='Country/Region', length=125)


[{'label': 'Global', 'value': 'Global'},
 {'label': 'Vietnam', 'value': 'Vietnam'},
 {'label': 'French Guiana', 'value': 'French Guiana'},
 {'label': 'Croatia', 'value': 'Croatia'},
 {'label': 'Cruise Ship', 'value': 'Cruise Ship'},
 {'label': 'Cuba', 'value': 'Cuba'},
 {'label': 'Cyprus', 'value': 'Cyprus'},
 {'label': 'Czechia', 'value': 'Czechia'},
 {'label': 'Denmark', 'value': 'Denmark'},
 {'label': 'Dominican Republic', 'value': 'Dominican Republic'},
 {'label': 'Ecuador', 'value': 'Ecuador'},
 {'label': 'Egypt', 'value': 'Egypt'},
 {'label': 'Estonia', 'value': 'Estonia'},
 {'label': 'Ethiopia', 'value': 'Ethiopia'},
 {'label': 'Finland', 'value': 'Finland'},
 {'label': 'France', 'value': 'France'},
 {'label': 'Georgia', 'value': 'Georgia'},
 {'label': 'Israel', 'value': 'Israel'},
 {'label': 'Germany', 'value': 'Germany'},
 {'label': 'Greece', 'value': 'Greece'},
 {'label': 'Guadeloupe', 'value': 'Guadeloupe'},
 {'label': 'Guinea', 'value': 'Guinea'},
 {'label': 'Guyana', 'valu

In [3]:
data.groupby('Country/Region').sum().iloc[:,-1].sort_values(ascending=False)

Country/Region
Vietnam            0.0
French Guiana      0.0
Croatia            0.0
Cruise Ship        0.0
Cuba               0.0
                  ... 
Nigeria            0.0
North Macedonia    0.0
Norway             0.0
Oman               0.0
Afghanistan        0.0
Name: 3/14/20, Length: 125, dtype: float64